# Factorization Machines on MovieLens

### Download ml-100k dataset

In [1]:
%%sh
wget http://files.grouplens.org/datasets/movielens/ml-100k.zip
unzip -o ml-100k.zip

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-100k/ub.test         


--2022-05-21 13:31:10--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

     0K .......... .......... .......... .......... ..........  1%  763K 6s
    50K .......... .......... .......... .......... ..........  2% 1.49M 5s
   100K .......... .......... .......... .......... ..........  3% 61.9M 3s
   150K .......... .......... .......... .......... ..........  4% 64.3M 2s
   200K .......... .......... .......... .......... ..........  5% 1.52M 2s
   250K .......... .......... .......... .......... ..........  6%  103M 2s
   300K .......... .......... .......... .......... ..........  7%  114M 2s
   350K .......... .......... .......... .......... ..........  8% 67.3M 1s
   400K .......... .......... ......

In [2]:
%cd ml-100k
!shuf ua.base -o ua.base.shuffled
!head -5 ua.base.shuffled

/home/ec2-user/SageMaker/ml-100k
870	732	2	882123355
86	242	4	879569486
303	68	4	879467361
551	765	1	892785194
404	1238	3	883790181


### Build training set and test set

In [3]:
num_users=943
num_movies=1682
num_features=num_users+num_movies

num_ratings_train=90570
num_ratings_test=9430

In [4]:
import csv
import numpy as np
from scipy.sparse import lil_matrix

def loadDataset(filename, lines, columns):
    # Features are one-hot encoded in a sparse matrix
    X = lil_matrix((lines, columns)).astype('float32')
    # Labels are stored in a vector
    Y = []
    line=0
    with open(filename,'r') as f:
        samples=csv.reader(f,delimiter='\t')
        for userId,movieId,rating,timestamp in samples:
            X[line,int(userId)-1] = 1
            X[line,int(num_users)+int(movieId)-1] = 1
            Y.append(int(rating))
            line=line+1       
    Y=np.array(Y).astype('float32')
    return X,Y

In [5]:
X_train, Y_train = loadDataset('ua.base.shuffled', num_ratings_train, num_features)
X_test, Y_test = loadDataset('ua.test', num_ratings_test, num_features)

In [6]:
print(X_train.shape)
print(Y_train.shape)
assert X_train.shape == (num_ratings_train, num_features)
assert Y_train.shape == (num_ratings_train, )

print(X_test.shape)
print(Y_test.shape)
assert X_test.shape  == (num_ratings_test, num_features)
assert Y_test.shape  == (num_ratings_test, )

(90570, 2625)
(90570,)
(9430, 2625)
(9430,)


### Convert to protobuf and save to S3

In [7]:
import sagemaker

sess   = sagemaker.Session()
bucket = sess.default_bucket()
prefix = 'fm-movielens'

train_key      = 'train.protobuf'
train_prefix   = '{}/{}'.format(prefix, 'train')

test_key       = 'test.protobuf'
test_prefix    = '{}/{}'.format(prefix, 'test')

output_prefix  = 's3://{}/{}/output'.format(bucket, prefix)

In [8]:
import io, boto3
import sagemaker.amazon.common as smac

def writeDatasetToProtobuf(X, Y, bucket, prefix, key):
    buf = io.BytesIO()
    smac.write_spmatrix_to_sparse_tensor(buf, X, Y)
    # use smac.write_numpy_to_dense_tensor(buf, feature, label) for numpy arrays
    buf.seek(0)
    print(buf)
    obj = '{}/{}'.format(prefix, key)
    boto3.resource('s3').Bucket(bucket).Object(obj).upload_fileobj(buf)
    return 's3://{}/{}'.format(bucket,obj)
    
train_data = writeDatasetToProtobuf(X_train, Y_train, bucket, train_prefix, train_key)    
test_data  = writeDatasetToProtobuf(X_test, Y_test, bucket, test_prefix, test_key)    
  
print(train_data)
print(test_data)
print('Output: {}'.format(output_prefix))

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


s3://sagemaker-us-east-1-608009264409/fm-movielens/train/train.protobuf
s3://sagemaker-us-east-1-608009264409/fm-movielens/test/test.protobuf
Output: s3://sagemaker-us-east-1-608009264409/fm-movielens/output


### Run training job

In [9]:
from sagemaker import get_execution_role
from sagemaker.image_uris import retrieve
from sagemaker.estimator import Estimator

region = sess.boto_session.region_name      
container = retrieve('factorization-machines', region)

In [15]:
fm = Estimator(
    container,
    role=get_execution_role(),
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=output_prefix)

fm.set_hyperparameters(
    feature_dim=num_features,
    predictor_type='regressor',
    num_factors=64,
    epochs=10)

fm.fit({'train': train_data, 'test': test_data})

2022-05-21 13:34:32 Starting - Starting the training job...
2022-05-21 13:34:55 Starting - Preparing the instances for trainingProfilerReport-1653140071: InProgress
.........
2022-05-21 13:36:16 Downloading - Downloading input data...
2022-05-21 13:36:56 Training - Downloading the training image.....Docker entrypoint called with argument(s): train
Running default environment configuration script
/opt/amazon/lib/python3.7/site-packages/jsonref.py:8: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, MutableMapping, Sequence
/opt/amazon/lib/python3.7/site-packages/algorithm/network_builder.py:87: DeprecationWarning: invalid escape sequence \s
  """
/opt/amazon/lib/python3.7/site-packages/algorithm/network_builder.py:120: DeprecationWarning: invalid escape sequence \s
  """
[05/21/2022 13:37:40 INFO 139950227810112] Reading default configurati

### Deploy model

In [16]:
endpoint_name = 'fm-movielens-100k'

fm_predictor = fm.deploy(
    endpoint_name=endpoint_name,
    instance_type='ml.t2.medium',
    initial_instance_count=1)

----------!

In [17]:
import json
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import JSONSerializer

class FMSerializer(JSONSerializer):
    def serialize(self, data):
       js = {'instances': []}
       for row in data:
              js['instances'].append({'features': row.tolist()})
       return json.dumps(js)

fm_predictor.serializer = FMSerializer()
fm_predictor.deserializer = JSONDeserializer()

### Run predictions

In [18]:
result = fm_predictor.predict(X_test[:3].toarray())
print(result)

{'predictions': [{'score': 3.377105236053467}, {'score': 3.41229248046875}, {'score': 3.6281182765960693}]}


In [19]:
fm_predictor.delete_endpoint()